In [28]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [2]:
import springtime
from springtime.datasets.modis import ModisSinglePoint, ModisMultiplePoints, modis_products, modis_bands, modis_dates

In [3]:
from pyPhenology import models, utils
import numpy as np

In [4]:
observations, predictors = utils.load_test_data(name='vaccinium', phenophase='flowers')

In [5]:
print(observations.shape)
observations.head()

(48, 5)


,species,site_id,year,doy,phenophase
48,vaccinium corymbosum,1,1998,122,501
49,vaccinium corymbosum,1,1998,122,501
50,vaccinium corymbosum,1,1991,124,501
51,vaccinium corymbosum,1,1991,124,501
52,vaccinium corymbosum,1,1998,126,501


In [17]:
print(observations["site_id"].unique())
print(observations["doy"].min())
print(observations["doy"].max())
print(observations["year"].min())
print(observations["year"].max())

[1]
122
144
1990
2001


In [6]:
predictors.head()

,site_id,temperature,year,doy,latitude,longitude,daylength
0,1,13.10,1990,-65,42.5429,-72.2011,10.24
1,1,13.26,1990,-64,42.5429,-72.2011,10.20
2,1,12.30,1990,-63,42.5429,-72.2011,10.16
3,1,12.15,1990,-62,42.5429,-72.2011,10.11
4,1,13.00,1990,-61,42.5429,-72.2011,10.07


In [16]:
print(predictors["site_id"].unique())
print(predictors["doy"].min())
print(predictors["doy"].max())
print(predictors["year"].min())
print(predictors["year"].max())

[1]
-65
297
1990
2001


In [8]:
ls_products = modis_products()
ls_products

,product,description,frequency,resolution_meters
1,Daymet,Daily Surface Weather Data (Daymet) on a 1-km ...,1 day,1000
2,ECO4ESIPTJPL,ECOSTRESS Evaporative Stress Index PT-JPL (ESI...,Varies,70
3,ECO4WUE,ECOSTRESS Water Use Efficiency (WUE) Daily L4 ...,Varies,70
4,GEDI03,GEDI Gridded Land Surface Metrics (LSM) L3 1km...,One time,1000
5,GEDI04_B,GEDI Gridded Aboveground Biomass Density (AGBD...,One time,1000
6,MCD12Q1,MODIS/Terra+Aqua Land Cover Type (LC) Yearly L...,1 year,500
7,MCD12Q2,MODIS/Terra+Aqua Land Cover Dynamics (LCD) Yea...,1 year,500
8,MCD15A2H,MODIS/Terra+Aqua Leaf Area Index/FPAR (LAI/FPA...,8 day,500
9,MCD15A3H,MODIS/Terra+Aqua Leaf Area Index/FPAR (LAI/FPA...,4 day,500
10,MCD19A3,MODIS/Terra+Aqua BRDF Model Parameters (MAIAC)...,8 day,1000


In [9]:
bands = modis_bands(product = "MCD12Q2")
bands

,band,description,units,valid_range,fill_value,scale_factor
1,Dormancy.Num_Modes_01,Onset_Dormancy,days since 1-1-1970,11138 to 32766,32767,1.0
2,Dormancy.Num_Modes_02,Onset_Dormancy,days since 1-1-1970,11138 to 32766,32767,1.0
3,EVI_Amplitude.Num_Modes_01,EVI_Amplitude,NBAR-EVI2,0 to 10000,32767,0.0001
4,EVI_Amplitude.Num_Modes_02,EVI_Amplitude,NBAR-EVI2,0 to 10000,32767,0.0001
5,EVI_Area.Num_Modes_01,EVI_Area,NBAR-EVI2,0 to 3700,32767,0.1
6,EVI_Area.Num_Modes_02,EVI_Area,NBAR-EVI2,0 to 3700,32767,0.1
7,EVI_Minimum.Num_Modes_01,Minimum_EVI,NBAR-EVI2,0 to 10000,32767,0.0001
8,EVI_Minimum.Num_Modes_02,Minimum_EVI,NBAR-EVI2,0 to 10000,32767,0.0001
9,Greenup.Num_Modes_01,Onset_Greenness_Increase,days since 1-1-1970,11138 to 32766,32767,1.0
10,Greenup.Num_Modes_02,Onset_Greenness_Increase,days since 1-1-1970,11138 to 32766,32767,1.0


In [33]:
df = pd.read_csv("combined_DBF.csv")
df['geometry'] = df['geometry'].apply(wkt.loads)
df = gpd.GeoDataFrame(df, crs='epsg:4326')
df['latitude'] = df.geometry.apply(lambda p: p.y)
df['longitude'] = df.geometry.apply(lambda p: p.x)
df = df.groupby(["site_id","latitude","longitude","first_yes_year"], as_index=False)["first_yes_doy"].mean()
df = df.rename(columns={"first_yes_year": "year", "first_yes_doy": "doy"})
df1 = df.head(20)
df1

,site_id,latitude,longitude,year,doy
0,2,43.085350,-70.691330,2009,113.285714
1,2,43.085350,-70.691330,2010,100.500000
2,2,43.085350,-70.691330,2011,110.666667
3,2,43.085350,-70.691330,2012,90.000000
4,2,43.085350,-70.691330,2013,112.833333
5,2,43.085350,-70.691330,2014,121.000000
6,2,43.085350,-70.691330,2015,121.000000
7,2,43.085350,-70.691330,2016,114.833333
8,2,43.085350,-70.691330,2017,110.000000
9,3,35.879211,-84.172203,2011,62.000000


In [37]:
dataset = ModisMultiplePoints(points=list(zip(df1.longitude.unique(), df1.latitude.unique())), 
                              years=[2007, 2017], product = "MCD12Q2", bands = ["Greenup.Num_Modes_01"])
dataset

ModisMultiplePoints(dataset='modis_multiple_points', points=[(-70.69133, 43.08535), (-84.172203, 35.879211), (-87.892998, 43.097801)], years=(2007, 2017), product='MCD12Q2', bands={'Greenup.Num_Modes_01'})

In [38]:
dataset.download()
df = dataset.load()
df.head()


R version 4.2.1 (2022-06-23) -- "Funny-Looking Kid"
Copyright (C) 2022 The R Foundation for Statistical Computing
Platform: aarch64-unknown-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

>             library(MODISTools)


Warning messages:
1: multiple methods tables found for ‘direction’ 
2: multiple methods tables found for ‘gridDistance’ 


>             mt_subset(product = "MCD12Q2",
+                 lat = 43.08535,
+                 lon = -70.69133,
+                 band = "Greenup.Num_Modes_01",
+                 start = "2007-01-01",
+                 end = "2017-12-31",
+                 km_lr = 0.0,
+                 km_ab = 0.0,
+                 site_name = "modis_-70.69133_43.08535",
+                 out_dir="/tmp/data",
+                 internal = FALSE,
+                 progress = FALSE)
NULL
>         
> 

R version 4.2.1 (2022-06-23) -- "Funny-Looking Kid"
Copyright (C) 2022 The R Foundation for Statistical Computing
Platform: aarch64-unknown-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation

Warning messages:
1: multiple methods tables found for ‘direction’ 
2: multiple methods tables found for ‘gridDistance’ 


NULL
>         
> 

R version 4.2.1 (2022-06-23) -- "Funny-Looking Kid"
Copyright (C) 2022 The R Foundation for Statistical Computing
Platform: aarch64-unknown-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

>             library(MODISTools)


Warning messages:
1: multiple methods tables found for ‘direction’ 
2: multiple methods tables found for ‘gridDistance’ 


>             mt_subset(product = "MCD12Q2",
+                 lat = 43.097801,
+                 lon = -87.892998,
+                 band = "Greenup.Num_Modes_01",
+                 start = "2007-01-01",
+                 end = "2017-12-31",
+                 km_lr = 0.0,
+                 km_ab = 0.0,
+                 site_name = "modis_-87.892998_43.097801",
+                 out_dir="/tmp/data",
+                 internal = FALSE,
+                 progress = FALSE)
NULL
>         
> 


,xllcorner,yllcorner,cellsize,nrows,ncols,band,units,scale,latitude,longitude,...,start,end,complete,modis_date,calendar_date,tile,proc_date,pixel,value,geometry
0,-5740907.9,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.08535,-70.69133,...,2007-01-01,2017-12-31,True,A2007001,2007-01-01,h12v04,2022085214812,1,13627,POINT (-70.69133 43.08535)
1,-5740907.9,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.08535,-70.69133,...,2007-01-01,2017-12-31,True,A2008001,2008-01-01,h12v04,2022095003237,1,13991,POINT (-70.69133 43.08535)
2,-5740907.9,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.08535,-70.69133,...,2007-01-01,2017-12-31,True,A2009001,2009-01-01,h12v04,2022096090426,1,14352,POINT (-70.69133 43.08535)
3,-5740907.9,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.08535,-70.69133,...,2007-01-01,2017-12-31,True,A2010001,2010-01-01,h12v04,2022096142536,1,14706,POINT (-70.69133 43.08535)
4,-5740907.9,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.08535,-70.69133,...,2007-01-01,2017-12-31,True,A2011001,2011-01-01,h12v04,2022101031346,1,15080,POINT (-70.69133 43.08535)


As the timing of land surface green-up in the MCD12Q2 dataset starts from July 1 in 1970, we need to convert the green-up dates to the units in day of year ranging from 1 to 365 in the corresponding year

In [40]:
df['year'] = pd.DatetimeIndex(df['calendar_date']).year
df['doy'] = df['value']-(df['year']-1970)*365
df

,xllcorner,yllcorner,cellsize,nrows,ncols,band,units,scale,latitude,longitude,...,complete,modis_date,calendar_date,tile,proc_date,pixel,value,geometry,year,doy
0,-5740907.90,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.085350,-70.691330,...,True,A2007001,2007-01-01,h12v04,2022085214812,1,13627,POINT (-70.69133 43.08535),2007,122
1,-5740907.90,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.085350,-70.691330,...,True,A2008001,2008-01-01,h12v04,2022095003237,1,13991,POINT (-70.69133 43.08535),2008,121
2,-5740907.90,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.085350,-70.691330,...,True,A2009001,2009-01-01,h12v04,2022096090426,1,14352,POINT (-70.69133 43.08535),2009,117
3,-5740907.90,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.085350,-70.691330,...,True,A2010001,2010-01-01,h12v04,2022096142536,1,14706,POINT (-70.69133 43.08535),2010,106
4,-5740907.90,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.085350,-70.691330,...,True,A2011001,2011-01-01,h12v04,2022101031346,1,15080,POINT (-70.69133 43.08535),2011,115
5,-5740907.90,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.085350,-70.691330,...,True,A2012001,2012-01-01,h12v04,2022103175437,1,15446,POINT (-70.69133 43.08535),2012,116
6,-5740907.90,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.085350,-70.691330,...,True,A2013001,2013-01-01,h12v04,2022105132233,1,15816,POINT (-70.69133 43.08535),2013,121
7,-5740907.90,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.085350,-70.691330,...,True,A2014001,2014-01-01,h12v04,2022110132941,1,16186,POINT (-70.69133 43.08535),2014,126
8,-5740907.90,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.085350,-70.691330,...,True,A2015001,2015-01-01,h12v04,2022111202223,1,16552,POINT (-70.69133 43.08535),2015,127
9,-5740907.90,4790653.45,463.312717,1,1,Greenup.Num_Modes_01,days since 1-1-1970,1.0,43.085350,-70.691330,...,True,A2016001,2016-01-01,h12v04,2022113062129,1,16919,POINT (-70.69133 43.08535),2016,129


In [41]:
df.shape

(33, 24)